**Create or get a PySpark instance**

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession \
        .builder \
        .master("local[11]") \
        .config("spark.executor.memory", "16g") \
        .config("spark.driver.memory", "16g") \
        .config("spark.memory.offHeap.enabled",True) \
        .config("spark.memory.offHeap.size","16g") \
        .appName("Airline") \
        .getOrCreate()

22/12/08 21:17:05 WARN Utils: Your hostname, LinuxGUI resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface eno1)
22/12/08 21:17:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/08 21:17:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.sparkContext.setLogLevel("WARN")

### Getting Pairs of Airport and Weather Stations ###

**Read flights, airports, and weather stations data.**

In [2]:
# All flights dataset
flights_df = spark.read.option("header",True).csv('../data/flights/')
# GHCND stations information
stations_df = spark.read.option("header",False).csv('../data/ghcnd-stations_clean.csv')
# United States airport information
airports_df = spark.read.option("header",True).csv('../data/us-airports_clean.csv')

# Create tables for these dataframe in order to query
flights_df.createOrReplaceTempView("Flights")
stations_df.createOrReplaceTempView("Stations")
airports_df.createOrReplaceTempView("Airports")

22/12/08 21:17:12 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
flights_df.head()

Row(FL_DATE='2013-01-01', OP_CARRIER='VX', OP_CARRIER_FL_NUM='108', ORIGIN='LAX', DEST='IAD', CRS_DEP_TIME='700', DEP_TIME='700.0', DEP_DELAY='0.0', TAXI_OUT='8.0', WHEELS_OFF='708.0', WHEELS_ON='1411.0', TAXI_IN='7.0', CRS_ARR_TIME='1445', ARR_TIME='1418.0', ARR_DELAY='-27.0', CANCELLED='0.0', CANCELLATION_CODE=None, DIVERTED='0.0', CRS_ELAPSED_TIME='285.0', ACTUAL_ELAPSED_TIME='258.0', AIR_TIME='243.0', DISTANCE='2288.0', CARRIER_DELAY=None, WEATHER_DELAY=None, NAS_DELAY=None, SECURITY_DELAY=None, LATE_AIRCRAFT_DELAY=None, Unnamed: 27=None)

In [8]:
flights_df.select(  
                    max(col("DEP_DELAY").cast('double')).alias('dep_max'), \
                    min(col("DEP_DELAY").cast('double')).alias('dep_min'), \
                    max(col("ARR_DELAY").cast('double')).alias('arr_max'), \
                    min(col("ARR_DELAY").cast('double')).alias('arr_min') \
                ).show()

+-------+-------+-------+-------+
|dep_max|dep_min|arr_max|arr_min|
+-------+-------+-------+-------+
| 2755.0| -251.0| 2692.0| -411.0|
+-------+-------+-------+-------+



**All airport codes in dataset.**

In [6]:
apCode_df = spark.sql("SELECT DISTINCT ORIGIN FROM Flights \
                        UNION \
                        SELECT DISTINCT DEST FROM Flights")
apCode_df.show()
apCode_df.createOrReplaceTempView("Airport_Code")

+------+
|ORIGIN|
+------+
|   BGM|
|   INL|
|   PSE|
|   DLG|
|   MSY|
|   PPG|
|   GEG|
|   DRT|
|   SNA|
|   BUR|
|   GRB|
|   GTF|
|   IFP|
|   IDA|
|   GRR|
|   LWB|
|   JLN|
|   PVU|
|   EUG|
|   PSG|
+------+
only showing top 20 rows



**All airline carriers in dataset**

In [5]:
spark.sql("SELECT DISTINCT OP_CARRIER FROM Flights").show()

+----------+
|OP_CARRIER|
+----------+
|        UA|
|        AA|
|        EV|
|        B6|
|        DL|
|        OO|
|        F9|
|        YV|
|        US|
|        MQ|
|        HA|
|        AS|
|        FL|
|        VX|
|        WN|
|        9E|
|        NK|
|        XE|
|        CO|
|        NW|
+----------+
only showing top 20 rows



**Append full airport information with airport codes appeared in airline delay and cancellation dataset**<br/>
columns: (ORIGIN, STATE, CITY, AP LOCAL_CODE, AP IATA_CODE, AP IDENT_CODE, AP NAME)

In [7]:
airport_full_df = spark.sql(" \
        SELECT c.ORIGIN, \
            p.local_region AS STATE, \
            p.municipality AS CITY, \
            p.local_code AS LOCAL, \
            p.iata_code AS IATA, \
            p.ident AS IDENT, \
            p.name AS NAME \
        FROM Airport_Code AS c \
        LEFT JOIN Airports AS p ON \
        (c.ORIGIN = p.local_code OR c.ORIGIN = p.iata_code)" \
    )
    
# airport_full_df.write.option("header",True).csv('../output/airports_full')
airport_full_df.createOrReplaceTempView("Airport_Full")

AnalysisException: path file:/home/ax/Develop/CS543/Project1/output/airports_full already exists.

### Appending Features to Airlines Dataset ###

**Load preprocessed airport-station dataset and GHCND weather data**

In [10]:
airport_station_df = spark.read.option("header",True).csv('../data/airports_stations.csv')
ghcnd_all = spark.read.option("header",True).csv('../data/filtered_weather.csv').drop('M_FLAG','Q_FLAG','S_FLAG','OBS_TIME')

In [5]:
print('Airport and Stations header:')
print(airport_station_df.head())
print('Stations daily data header:')
print(ghcnd_all.head())

Airport and Stations header:
Row(ORIGIN='BGM', STATE='NY', CITY='Binghamton', LOCAL='BGM', IATA='BGM', IDENT='KBGM', NAME='Greater Binghamton/Edwin A Link field', STATION='USW00014738')
Stations daily data header:
Row(STATION_ID='AQW00061705', AIRPORT_CODE='PPG', DATE='20090101', ELEMENT='TMAX', DATA_VALUE='322')


**Assign each existing airport codes into weather data value and filter out weather data that not relate to our dataset.**

In [11]:
selected_airport_station = airport_station_df.select(col('ORIGIN'),col('STATION'))
ghcnd_all = ghcnd_all.join(selected_airport_station, \
                                        ghcnd_all.STATION_ID == selected_airport_station.STATION, \
                                        "LeftOuter")

In [17]:
ghcnd_all.head()

Row(STATION_ID='AQW00061705', AIRPORT_CODE='PPG', DATE='20090101', ELEMENT='TMAX', DATA_VALUE='322', ORIGIN='PPG', STATION='AQW00061705')

**Ouput filtered weather dataset** <br />
Keep not null rows and rename them based on GHCND documents

In [ ]:
ghcnd_all.filter(ghcnd_all.ORIGIN.isNotNull()) \
        .select(col('_c0').alias("STATION_ID"), \
                col('ORIGIN').alias("AIRPORT_CODE"), \
                col('_c1').alias("DATE"), \
                col('_c2').alias("ELEMENT"), \
                col('_c3').alias("DATA_VALUE")) \
        .write.option("header",True).csv("../output/filtered_ghcnd")

# ghcnd_all.filter(ghcnd_all.ORIGIN.isNotNull()) \
#         .select(col('_c0').alias("STATION_ID"), \
#                 col('ORIGIN').alias("AIRPORT_CODE"), \
#                 col('_c1').alias("DATE"), \
#                 col('_c2').alias("ELEMENT"), \
#                 col('_c3').alias("DATA_VALUE"), \
#                 col('_c4').alias("M_FLAG"), \
#                 col('_c5').alias("Q_FLAG"), \
#                 col('_c6').alias("S_FLAG"), \
#                 col('_c7').alias("OBS_TIME")) \
#         .write.option("header",True).csv("./output/filtered_weatherData")

***Four extreme points in mainland of the United States***<br />
Sumas, Washington 49°00′08.6″N 122°15′40″W – northernmost incorporated place in the 48 contiguous states<br />
Key West, Florida 24°32′38.724″N 81°48′17.658″W – southernmost incorporated place in the contiguous 48 states<br />
West Quoddy Head, Maine 44°48′55.4″N 66°56′59.2″W – easternmost point on the U.S. mainland<br />
Port Orford, Oregon 42.754065°N 124.512605°W – westernmost incorporated place in the 48 contiguous states<br />

**Variables**

In [9]:
# top_left corner has both greater latitude and greater longitude
# Northeast,Southeast,Mid-Northeast,Mid-Southeast,Mid-Northwest,Mid-Southwest,Northwest,Southwest
# [Top_left(lat,lon), Bottom_right(lat,lon)]
SE = [(37,80),(24,65)]
NE = [(50,80),(37,65)]
MSE = [(37,95),(24,80)]
MNE = [(50,95),(37,80)]
MSW = [(37,110),(24,95)]
MNW = [(50,110),(37,95)]
SW = [(37,125),(24,110)]
NW = [(50,125),(37,110)]

**Add geological coordinates for each station**

In [12]:
full_stations_info = spark.read.option("header",False).csv('../data/ghcnd-stations.csv')

**Load geological info for each airport and station**

In [13]:
geo = airport_station_df \
    .join(full_stations_info,airport_station_df.STATION == full_stations_info._c0,'leftouter') \
    .select(col('ORIGIN').alias("AIRPORT_CODE"), \
            col('STATE'), \
            col('CITY'), \
            col('NAME'), \
            col('STATION'), \
            col('_c1').alias('LAITITUDE').cast('Double'), \
            col('_c2').alias('LONGITUDE').cast('Double') \
        ).withColumn('LONGITUDE', col('LONGITUDE')*-1)

**Check max and min of LAITITUDE and LONGITUDE**

In [14]:
geo.select( \
            max(col('LAITITUDE')), \
            min(col('LAITITUDE')), \
            max(col('LONGITUDE')), \
            min(col('LONGITUDE')), \
        ).write.option('header',True).csv('../output/geo_max_min')

__Assign area for each airport/station__

In [15]:
areas = geo.withColumn("AREA",when((geo.LAITITUDE < SE[0][0]) & (geo.LONGITUDE < SE[0][1]),"SE") \
                                                .when((geo.LAITITUDE < NE[0][0]) & (geo.LONGITUDE < NE[0][1]),"NE") \
                                                .when((geo.LAITITUDE < MSE[0][0]) & (geo.LONGITUDE < MSE[0][1]),"MSE") \
                                                .when((geo.LAITITUDE < MNE[0][0]) & (geo.LONGITUDE < MNE[0][1]),"MNE") \
                                                .when((geo.LAITITUDE < MSW[0][0]) & (geo.LONGITUDE < MSW[0][1]),"MSW") \
                                                .when((geo.LAITITUDE < MNW[0][0]) & (geo.LONGITUDE < MNW[0][1]),"MNW") \
                                                .when((geo.LAITITUDE < SW[0][0]) & (geo.LONGITUDE < SW[0][1]),"SW") \
                                                .when((geo.LAITITUDE < NW[0][0]) & (geo.LONGITUDE < NW[0][1]),"NW") \
                                                .when((geo.LAITITUDE > NW[0][0]) & (geo.LONGITUDE > NW[0][1]),"NW") \
                                                .when((geo.LAITITUDE < SW[0][0]) & (geo.LONGITUDE > SW[0][1]),"SW") \
                                                .when((geo.LAITITUDE < SE[0][0]) & (geo.LONGITUDE < SE[0][1]),"SE") \
                                                .when((geo.LAITITUDE > NE[0][0]) & (geo.LONGITUDE < NE[0][1]),"NE") \
                                                .otherwise(geo.STATION)) \
                            .drop('LAITITUDE','LONGITUDE')

In [ ]:
areas.write.option('header',True).csv('../output/areas')

### Testing data appending for flights in 2018 ###

**Load preprocessed airport-station dataset and GHCND weather data**

In [16]:
airports_stations =  spark.read.option("header",True).csv('../data/airports_stations.csv')
# ghcnd2018 = spark.read.option("header",False).csv('./data/filtered_weather.csv')
ghcnd2018 = ghcnd_all
fl2018 = spark.read.option("header",True).csv('../data/flights/2018.csv')

In [15]:
ghcnd2018.head()

Row(STATION_ID='AQW00061705', AIRPORT_CODE='PPG', DATE='20090101', ELEMENT='TMAX', DATA_VALUE='322', ORIGIN='PPG', STATION='AQW00061705')

**Assign each existing airport codes into weather data value and filter out weather data that not relate to our dataset.** <br/>
**Keep not null rows and rename them based on GHCND documents**

In [17]:
ghcnd2018 = ghcnd2018.join(airports_stations, \
                            ghcnd2018._c0 == airports_stations.STATION, \
                            "LeftOuter")
ghcnd2018 = ghcnd2018.filter(ghcnd2018.ORIGIN.isNotNull()) \
                                .select(col('_c0').alias("STATION_ID"), \
                                        col('ORIGIN').alias("AIRPORT_CODE"), \
                                        col('_c1').alias("DATE"), \
                                        col('_c2').alias("ELEMENT"), \
                                        col('_c3').alias("DATA_VALUE"), \
                                        col('_c4').alias("M_FLAG"), \
                                        col('_c5').alias("Q_FLAG"), \
                                        col('_c6').alias("S_FLAG"), \
                                        col('_c7').alias("OBS_TIME"))

AttributeError: 'DataFrame' object has no attribute '_c0'

**Filter wanted features and join area category for ORIGIN and DEST**

In [18]:
df = fl2018.select(['FL_DATE','OP_CARRIER','OP_CARRIER_FL_NUM','ORIGIN','DEST','CRS_DEP_TIME','DEP_DELAY','CRS_ARR_TIME','ARR_DELAY'])

In [19]:
df = df.join(areas, fl2018.ORIGIN == areas.AIRPORT_CODE,'LeftOuter') \
                .drop('AIRPORT_CODE','STATE','CITY','NAME') \
                .withColumnRenamed('AREA', 'ORIG_AREA').drop('ORIGIN') \
                .withColumnRenamed('STATION', 'ORIG_STATION') \
        .join(areas, fl2018.DEST == areas.AIRPORT_CODE,'LeftOuter') \
                .drop('AIRPORT_CODE','STATE','CITY','NAME') \
                .withColumnRenamed('AREA', 'DEST_AREA').drop('DEST') \
                .withColumnRenamed('STATION', 'DEST_STATION')

**Append weather info according to ORIGIN stations and DEST stations**

In [20]:
df = df.join(ghcnd2018,(df.ORIG_STATION == ghcnd2018.STATION_ID) & (ghcnd2018.DATE == date_format(df.FL_DATE,"yyyyMMdd")),'Inner') \
                .drop('STATION','AIRPORT_CODE','DATE','STATION_ID') \
                .withColumnRenamed('ELEMENT', 'ORIG_WEATHER') \
                .withColumnRenamed('DATA_VALUE', 'ORIG_WEATHER_DATA') \
        .join(ghcnd2018, (df.DEST_STATION == ghcnd2018.STATION_ID) & (ghcnd2018.DATE == date_format(df.FL_DATE,"yyyyMMdd")),'Inner') \
                .drop('STATION','AIRPORT_CODE','DATE','STATION_ID') \
                .withColumnRenamed('ELEMENT', 'DEST_WEATHER') \
                .withColumnRenamed('DATA_VALUE', 'DEST_WEATHER_DATA') \
        .filter(((col('ORIG_WEATHER_DATA').cast('Double') != 0) & (col('ORIG_WEATHER').rlike('(PRCP|SNOW|SNWD|^WD+|^WS+)'))) | \
                ((col('DEST_WEATHER_DATA').cast('Double') != 0) & (col('DEST_WEATHER').rlike('(PRCP|SNOW|SNWD|^WD+|^WS+)')))) \
        .drop('ORIG_WEATHER_DATA','DEST_WEATHER_DATA')

In [21]:
df = df.withColumn('ORIG_WEATHER', when(~df.ORIG_WEATHER.rlike('(PRCP|SNOW|SNWD|^WD+|^WS+)'),"SUNNY") \
                                    .otherwise(df.ORIG_WEATHER)) \
        .withColumn('DEST_WEATHER', when(~df.DEST_WEATHER.rlike('(PRCP|SNOW|SNWD|^WD+|^WS+)'),"SUNNY") \
                                    .otherwise(df.DEST_WEATHER))

In [22]:
df = df.groupBy('FL_DATE', \
                'OP_CARRIER', \
                'OP_CARRIER_FL_NUM', \
                'CRS_DEP_TIME', \
                'DEP_DELAY', \
                'CRS_ARR_TIME', \
                'ARR_DELAY', \
                'ORIG_STATION', \
                'ORIG_AREA', \
                'DEST_STATION', \
                'DEST_AREA') \
        .agg(collect_list('ORIG_WEATHER').alias('ORIG_WEATHERS'), collect_list('DEST_WEATHER').alias('DEST_WEATHERS'))

**Chose Represented Weather**

In [23]:
df = df.withColumn('ORIG_WEATHER', when((array_contains(df.ORIG_WEATHERS,'SNWD') | array_contains(df.ORIG_WEATHERS,'SNOW') | array_contains(df.ORIG_WEATHERS,'WT18')),'Snow') \
                                    .when((array_contains(df.ORIG_WEATHERS,'PRCP') | array_contains(df.ORIG_WEATHERS,'WT16')),'Rain') \
                                    .when((array_contains(df.ORIG_WEATHERS,'WDF2') | array_contains(df.ORIG_WEATHERS,'WDF5') | array_contains(df.ORIG_WEATHERS,'WT11')),'Wind') \
                                    .otherwise('Sunny')) \
        .withColumn('DEST_WEATHER', when((array_contains(df.DEST_WEATHERS,'SNWD') | array_contains(df.DEST_WEATHERS,'SNOW') | array_contains(df.DEST_WEATHERS,'WT18')),'Snow') \
                                    .when((array_contains(df.DEST_WEATHERS,'PRCP') | array_contains(df.DEST_WEATHERS,'WT16')),'Rain') \
                                    .when((array_contains(df.DEST_WEATHERS,'WDF2') | array_contains(df.DEST_WEATHERS,'WDF5') | array_contains(df.DEST_WEATHERS,'WT11')),'Wind') \
                                    .otherwise('Sunny'))

In [24]:
df = df.drop('ORIG_WEATHERS','DEST_WEATHERS')

### Other features' category:


In [25]:
full_stations_info = spark.read.option("header",False).csv('../data/ghcnd-stations.csv')

**Apply busy/not busy categories for ORIG and DEST**

In [26]:
df = df.withColumn('ORIG_SCHEDULE', \
                    when((df.CRS_DEP_TIME.cast('Integer') >= 800) & (df.CRS_DEP_TIME < 1800),"Most Busy") \
                    .when((df.CRS_DEP_TIME.cast('Integer') >= 600) & (df.CRS_DEP_TIME < 800),"Least Busy") \
                    .otherwise("Busy")) \
        .withColumn('DEST_SCHEDULE', \
                    when((df.CRS_ARR_TIME.cast('Integer') >= 800) & (df.CRS_ARR_TIME < 1800),"Most Busy") \
                    .when((df.CRS_ARR_TIME.cast('Integer') >= 600) & (df.CRS_ARR_TIME < 800),"Least Busy") \
                    .otherwise("Busy")) \
        .drop('CRS_DEP_TIME','CRS_ARR_TIME')

**Apply carrier type categories**

In [27]:
carriers = spark.read.option("header",True).csv('../data/carriers_type.csv')

df = df.join(carriers, df.OP_CARRIER == carriers.OP_CARRIER, 'LeftOuter') \
        .drop('OP_CARRIER') \
        .withColumnRenamed('CATEGORY','CARRIER_TYPE')

**Apply delay categories for DEPART and ARRIVE**

In [28]:
df = df.withColumn('DEP_DELAY_CAT', 
                when((df.DEP_DELAY.cast('Double') < -60),"60+min early") \
                .when((df.DEP_DELAY.cast('Double') >= -60) & (df.DEP_DELAY < -30),"30~60min early") \
                .when((df.DEP_DELAY.cast('Double') >= -30) & (df.DEP_DELAY < -15),"15~30min early") \
                .when((df.DEP_DELAY.cast('Double') >= -15) & (df.DEP_DELAY < -5),"5~15min early") \
                .when((df.DEP_DELAY.cast('Double') >= -5) & (df.DEP_DELAY <= 5),"On Time") \
                .when((df.DEP_DELAY.cast('Double') > 5) & (df.DEP_DELAY <= 15),"5~15min late") \
                .when((df.DEP_DELAY.cast('Double') > 15) & (df.DEP_DELAY <= 30),"15~30min late") \
                .when((df.DEP_DELAY.cast('Double') > 30) & (df.DEP_DELAY <= 60),"30~60min late") \
                .when((df.DEP_DELAY.cast('Double') > -60),"60+min late") \
                .otherwise("On Time")) \
    .withColumn('ARR_DELAY_CAT', 
                when((df.ARR_DELAY.cast('Double') < -60),"60+min early") \
                .when((df.ARR_DELAY.cast('Double') >= -60) & (df.ARR_DELAY < -30),"30~60min early") \
                .when((df.ARR_DELAY.cast('Double') >= -30) & (df.ARR_DELAY < -15),"15~30min early") \
                .when((df.ARR_DELAY.cast('Double') >= -15) & (df.ARR_DELAY < -5),"5~15min early") \
                .when((df.ARR_DELAY.cast('Double') >= -5) & (df.ARR_DELAY <= 5),"On Time") \
                .when((df.ARR_DELAY.cast('Double') > 5) & (df.ARR_DELAY <= 15),"5~15min late") \
                .when((df.ARR_DELAY.cast('Double') > 15) & (df.ARR_DELAY <= 30),"15~30min late") \
                .when((df.ARR_DELAY.cast('Double') > 30) & (df.ARR_DELAY <= 60),"30~60min late") \
                .when((df.ARR_DELAY.cast('Double') > -60),"60+min late") \
                .otherwise("On Time")) \
    .drop('DEP_DELAY','ARR_DELAY')

**Drop useless data and Output the csv file**

In [20]:
df = df.drop('FL_DATE','OP_CARRIER_FL_NUM','ORIG_STATION','DEST_STATION','ORIG_WEATHERS','DEST_WEATHERS')

df.coalesce(1).write.option('header',True).csv('../output/2018')